In [13]:
import sqlite3
import requests
import time
from bs4 import BeautifulSoup


In [ ]:
def scrape_google_repos(max_pages=1):
    
    base_url = "https://github.com/orgs/google/repositories"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept-Language": "en-US,en;q=0.9",
    }

    all_repos = []

    for page in range(1, max_pages + 1):
        url = f"{base_url}?page={page}"
        print(f"[INFO] Fetching: {url}")

        resp = requests.get(url, headers=headers)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")

        h3_list = soup.find_all("h3")
        if not h3_list:
            print("[WARN] no h3 found")
            break

        for h3 in h3_list:
            a_tag = h3.find("a")
            if not a_tag:
                continue

            href = a_tag.get("href", "")
            if not href.startswith("/google/"):
                continue

            repo_name = a_tag.get_text(strip=True)

            container = h3.find_parent("li")
            if container is None:
                container = h3.parent


        #language
            language = None
            lang_span = container.find(
                "span",
                class_=lambda x: x and x.startswith("ReposListItem-module__Text_4--")
            )
            if lang_span:
                language = lang_span.get_text(strip=True)

           
            star_tag = container.find(
                "a",
                href=lambda x: x and x.endswith("/stargazers")
            )

        #star
            stars = None
            if star_tag:
                stars = star_tag.get_text(strip=True) 

            all_repos.append(
                {
                    "name": repo_name,
                    "language": language,
                    "stars": stars,
                }
            )

        time.sleep(1)  

    return all_repos


In [35]:
path = ''          # DBファイルの保存先パス（相対パスで指定）
db_name = 'github_repos.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQLを実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # テーブルの作成
    # name: リポジトリ名, language: 主な言語, stars: スター数
    sql = '''
    CREATE TABLE IF NOT EXISTS repos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        language TEXT,
        stars INTEGER
    );
    '''

    # SQL文の実行
    cur.execute(sql)

    # 変更をDBに反映させる
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()


In [38]:
path = ''
db_name = 'github_repos.db'

try:
    # 1. GitHubからデータをスクレイピング
    repos = scrape_google_repos(max_pages=94)  # まずは1ページだけ

    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQLを実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # 既存データを一旦削除してから挿入する場合
    cur.execute("DELETE FROM repos;")

    # INSERT文（プレースホルダを使う）
    sql = "INSERT INTO repos (name, language, stars) VALUES (?, ?, ?);"

    # 複数レコードをまとめて挿入
    data = [(r["name"], r["language"], r["stars"]) for r in repos]
    cur.executemany(sql, data)

    # 変更をDBに反映させる
    conn.commit()

    print(f"[INFO] {len(repos)} 件のデータを挿入しました。")

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()


[INFO] Fetching: https://github.com/orgs/google/repositories?page=1
[INFO] Fetching: https://github.com/orgs/google/repositories?page=2
[INFO] Fetching: https://github.com/orgs/google/repositories?page=3
[INFO] Fetching: https://github.com/orgs/google/repositories?page=4
[INFO] Fetching: https://github.com/orgs/google/repositories?page=5
[INFO] Fetching: https://github.com/orgs/google/repositories?page=6
[INFO] Fetching: https://github.com/orgs/google/repositories?page=7
[INFO] Fetching: https://github.com/orgs/google/repositories?page=8
[INFO] Fetching: https://github.com/orgs/google/repositories?page=9
[INFO] Fetching: https://github.com/orgs/google/repositories?page=10
[INFO] Fetching: https://github.com/orgs/google/repositories?page=11
[INFO] Fetching: https://github.com/orgs/google/repositories?page=12
[INFO] Fetching: https://github.com/orgs/google/repositories?page=13
[INFO] Fetching: https://github.com/orgs/google/repositories?page=14
[INFO] Fetching: https://github.com/orgs/go

In [39]:
path = ''
db_name = 'github_repos.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQLを実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # データを参照するSQL
    sql = "SELECT name, language, stars FROM repos ORDER BY stars DESC;"

    # SQL文の実行
    cur.execute(sql)

    # 実行結果の取得
    rows = cur.fetchall()

    # 結果の表示
    print("=== Google GitHub Repositories ===")
    for name, lang, stars in rows:
        print(f"名前: {name:40s} | 言語: {lang or '-':10s} | ⭐ {stars}")

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()


=== Google GitHub Repositories ===
名前: benchmark                                | 言語: C++        | ⭐ 9.9k
名前: magika                                   | 言語: Python     | ⭐ 9.8k
名前: go-cloud                                 | 言語: Go         | ⭐ 9.8k
名前: re2                                      | 言語: C++        | ⭐ 9.5k
名前: diff-match-patch                         | 言語: Python     | ⭐ 8k
名前: pprof                                    | 言語: Go         | ⭐ 8.9k
名前: tsunami-security-scanner                 | 言語: Java       | ⭐ 8.5k
名前: trax                                     | 言語: Python     | ⭐ 8.3k
名前: traceur-compiler                         | 言語: JavaScript | ⭐ 8.2k
名前: osv-scanner                              | 言語: Go         | ⭐ 8.1k
名前: gops                                     | 言語: Go         | ⭐ 7k
名前: draco                                    | 言語: C++        | ⭐ 7k
名前: accompanist                              | 言語: Kotlin     | ⭐ 7.8k
名前: model-viewer                             | 言